In [1]:
import json
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import os
from pycocotools import coco
import xml.etree.ElementTree as ET
import xml
from pycocotools import mask as comask
import cv2
from utils import DateCaptured, coco_bbox_creator, iou_calculator, mask2poly, binary_mask_to_rle
import datetime
import itertools
# import csv ozan changed

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
### CATEGORY INFORMATION

# Add the name of the folders inside the dataset and
# assign some label id. Start iding from 91.

dict_of_categories = {'face': 91}

In [3]:
### DATASET INFORMATION

# Enter the original/custom JSON file to be used.
# I used the original train2014 JSON file containing
# the instances.

# json_dir = "Datasets/coco/annotations_old/instances_train2014.json"

# for the purpose of creating validation images json
# json_dir = "Datasets/coco/annotations_old/instances_val2014.json" 

# for the purpose of creating testing images json
json_dir = "Datasets/coco/annotations_old/instances_minival2014.json" 

### For viewing the content of the new json file. ###
# json_dir = "Datasets/coco/annotations/instances_train2014.json"

### minival2014 will become the main dictionary ###
minival2014 = json.load(open(json_dir))

In [4]:
### CLEARING THE DATASET
# Use that if you only going to make a dataset of your own.
# If you are going to append the images, this operation is not necessary.

minival2014['images'] = []
minival2014['annotations'] = []
minival2014['categories'] = []

In [5]:
# Viewing the number of instances per class and gathering some statistics about them.
# labels = np.zeros((91, ), dtype=np.int32)
# for annotation in minival2014['annotations']:
#     labels[annotation['category_id']] += 1
# used_label_ids = np.where(labels!=0)
# ex_labels = labels[used_label_ids]
# print(ex_labels)
# print("Average instances per label: {}".format(sum(ex_labels)/81))
# print("Maximum instances per label: {}".format(max(ex_labels)))
# print("Minimum instances per label: {}".format(min(ex_labels)))

In [6]:
### APPENDING CLASSES TO THE CATEGORIES

### For viewing the categories inside the JSON file.
# minival2014['categories']

# Add the classes based on to the dict_of_categories
# Below shows an example for adding the face class to the main dictionary. 

minival2014['categories'].append({'id': 91, 'name': 'face', 'supercategory': 'person'})



In [7]:
# Some functions to be used:
def isjpg(string):
    if string[-4:]==".jpg":
        return True 
    
def ispng(string):
    if string[-4:]==".png":
        return True
    
def save_bboxes(image_name, bbox_coor):
    with open('img_bbox.txt','a+') as file:
        file.write(str(image_name)+','+str(bbox_coor[0])+','+str(bbox_coor[1])+','+str(bbox_coor[2])+','+str(bbox_coor[3])+'\n')

def xml_retreiver(tree_object, key_name):
    for iterator in tree_object.iter(key_name):
        return iterator.text

In [9]:
# Storing the data in memory and then writes them into a file. 

# from decimal import Decimal
from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.2f')

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

# with open("Datasets/coco/annotations/instances_train2014.json","w") as f:
    
with open("Datasets/coco/annotations/instances_minival2014.json","w") as f:
    data = json.dumps(minival2014, cls=MyEncoder, indent=4)
    f.write(data)
